In [92]:
# Exploring ebay car sales data. 
# dataset source: https://www.kaggle.com/orgesleka/used-cars-database/data#autos.csv

import pandas as pd
import numpy as np

autos = pt.read_csv('autos.csv',encoding='Latin-1')
# autos.info()
autos.head()
# the dataset contains 20 columns and most of them are strings. 
# columns contain none values

autos.columns
autos_dtypes = autos.dtypes
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gear_box', 'power_PS', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_pictures', 'postal_code',
       'last_seen']
autos.head()
# all camelcase converted to snakecase

autos['price'] = autos['price'].str.replace('$','').str.replace(',','').astype(float)
autos['odometer']= autos['odometer'].str.replace('km','').str.replace(',','').astype(float)
autos = autos.rename(columns={'odometer':'odometer_km'})
# autos.info()
# changed the price and odometer type to numbers/float
autos.price.unique().shape
autos.odometer_km.unique().shape
autos.price.describe()
autos.odometer_km.describe()
autos.price.value_counts().sort_index()
autos.odometer_km.value_counts().sort_index()

autos = autos[autos['price'].between(1,300000)]
# removed any number smaller than 1 and bigger than 300000

autos[['date_cr','time_cr']] = autos['date_crawled'].str.split(expand=True)
autos['date_cr'].value_counts(normalize=True,dropna=False).sort_index()

autos[['date_ad','time_ad']]= autos['ad_created'].str.split(expand=True)
autos['date_ad'].value_counts(normalize=True,dropna=False).sort_index()

autos[['date_la','time_la']]= autos['last_seen'].str.split(expand=True)
autos['date_la'].value_counts(normalize=True,dropna=False).sort_index()
# create a percentage expression based on the date of 'date_crawled', 'ad_created' and 'last_seen.
autos.registration_year.value_counts().sort_index()

#selecting registration_year within 1900 to 2016
autos = autos[autos['registration_year'].between(1900,2016)]
percentage_registration_within_20years=(autos['registration_year'].value_counts(normalize=True).sort_index(ascending = False).head(20))

sum(percentage_registration_within_20years)
# about 87% registration occurs within the recent 20 years

bp = {}
car_brand_counts = autos['brand'].value_counts(normalize=True)
common_brand = car_brand_counts[car_brand_counts >0.05].index
print(common_brand)
for b in common_brand:
    selected_rows = autos[autos['brand'] == b] 
    car_brand = selected_rows['brand'].iloc[0]
    mean_price = selected_rows['price'].mean()
    bp[b]=mean_price
bp_series = pd.Series(bp)
df1=pd.DataFrame(bp_series, columns=['mean_price'])

# for all the car_brand_counts bigger than 5% and calculate their mean values

bm ={}
for b in common_brand:
    selected_rows = autos[autos['brand'] == b] 
    car_brand = selected_rows['brand'].iloc[0]
    mean_mileage = selected_rows['odometer_km'].mean()
    bm[b]=mean_mileage
bm_series = pd.Series(bm)
df2=pd.DataFrame(bm_series, columns=['mean_mileage'])

pd.concat([df1, df2], axis=1)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


,mean_price,mean_mileage
audi,9336.687454,129157.386785
bmw,8332.820518,132572.513140
ford,3749.469507,124266.012872
mercedes_benz,8628.450366,130788.363313
opel,2975.241935,129310.035842
volkswagen,5402.410262,128707.158791
